# Projet 8 - Openclassrooms - Déployer un modèle dans le cloud

# Pre-processing DEEP FEATURES

Echantillon d'images stocké sur compartiment S3 

## Import

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import Row, SparkSession, SQLContext
from pyspark.sql.functions import col, pandas_udf, PandasUDFType

import io
from io import StringIO
import os
import sys
import time

import numpy as np
import pandas as pd

import cv2
from PIL import Image


import boto.s3
import boto3


import sagemaker
from sagemaker import get_execution_role
import sagemaker_pyspark


In [248]:
import pickle
import cloudpickle
import pyspark.serializers
pyspark.serializers.cloudpickle = cloudpickle

In [86]:
pip install Keras

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install tensorflow

     |████████████████████████        | 295.3 MB 42.7 MB/s eta 0:00:03    |█▏                              | 13.9 MB 15.7 MB/s eta 0:00:25     |███▋                            | 44.2 MB 15.7 MB/s eta 0:00:23     |█████████▏                      | 112.4 MB 35.0 MB/s eta 0:00:09     |█████████▉                      | 120.6 MB 35.0 MB/s eta 0:00:08     |██████████▎                     | 127.1 MB 35.0 MB/s eta 0:00:08     |███████████▌                    | 141.5 MB 35.0 MB/s eta 0:00:08     |███████████████████             | 234.8 MB 61.9 MB/s eta 0:00:03

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |█████████████████████████████   | 358.6 MB 84.2 MB/s eta 0:00:01

     |█████████████████████████████▍  | 362.0 MB 84.2 MB/s eta 0:00:01

     |█████████████████████████████▋  | 364.5 MB 84.2 MB/s eta 0:00:01

...

     |██████████████████████████████▉ | 380.1 MB 84.2 MB/s eta 0:00:01

     |███████████████████████████████▏| 384.7 MB 84.2 MB/s eta 0:00:01

     |███████████████████████████████▋| 389.5 MB 84.2 MB/s eta 0:00:01

     |████████████████████████████████| 394.3 MB 84.2 MB/s eta 0:00:01

     |████████████████████████████████| 394.3 MB 6.0 kB/s 
     |████████████████████████████████| 3.8 MB 47.6 MB/s eta 0:00:01
     |████████████████████████████████| 10.6 MB 38.4 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 1.9 MB/s  eta 0:00:01
     |████████████████████████████████| 127 kB 69.8 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 5.7 MB/s  eta 0:00:01
     |████████████████████████████████| 462 kB 47.9 MB/s eta 0:00:01
     |████████████████████████████████| 136 kB 70.6 MB/s eta 0:00:01
     |████████████████████████████████| 781 kB 42.0 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 9.9 MB/s  eta 0:00:01
     |████████████████████████████████| 155 kB 69.3 MB/s eta 0:00:01
     |████████████████████████████████| 147 kB 69.7 MB/s eta 0:00:01
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4830 sha256=67f8ded58b5461724ff005f3287e94dbefdf4094b7d2e182c8a9b17502679f5d
  Stored in directory: /home/

In [2]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing import image

from pyspark.sql.functions import col, pandas_udf, PandasUDFType

# Définition des fonctions de pré-processing

In [235]:
# Renvoie la catéagorie d'une image à partir de son chemin  
# Entrée : chaine de caractères, contient le chemin complet de l'image 
# Sortie: tuple de chaines de caratères  catégorie, nom du fichier

def parse_categorie(path):
    if len(path) > 0:
        return path.split('/')[-1],path.split('/')[-2],
    else:
        return 'None','None'
    

# Renvoie les image format binaire converties en array
# Entrée: image lue en binaire dans le rdd
# Sortie: ligne tuple categorie, matrice de l'image (200x200)

def cat_image_np(imgfile_imgbytes):

    imgfilename, imgbytes = imgfile_imgbytes

    nom, categorie = parse_categorie(imgfilename)

    img = Image.open(io.BytesIO(imgbytes))

    img_np = img_to_array(img)

    return Row(fileName=nom,category=categorie, image=img_np.tolist())
    #return nom,categorie,img_np
    
# Renvoie les deep features resnet50
# Entrée: Ligne d'un RDD contenant ligne tuple categorie, matrice de l'image (200x200)
# Sortie: Ligne de RDD contenant nom (chaine), catégorie (chaine), vecteur sous forme d'une liste de 2048 reels  
def deep_features(y):
    var = np.array(y['image'])
    var = np.expand_dims(var, axis=0)
    var = preprocess_input(var)
    model = ResNet50(weights='imagenet', pooling='avg', include_top = False) 
    essai = model.predict(var) 
    essai = essai.squeeze() 
    return Row(category=y['category'],fileName=y['fileName'],feature=essai.tolist())  


# Ecrit un dataframe spark au format parket
# Entrée: dataframme spark, chemin (chaine)
# Sortie: sauvegarde format parquet sur l'emplacement indiqué

def write_dataframe_parquet(dataframe, path_parquet):
    '''Enregistrement du spark dataframe au format parquet au chemin specifie'''
    try:
        #start = time.time()
        dataframe.write.format("parquet").mode('overwrite').save(path_parquet)
        print('Enregistrement effectué.')
            

        #print('Temps de sauvegarde : {} secondes'.format(time.strftime('%S', time.gmtime(time.time()-start))))
    except:
        print('L\'Enregistrement a échoué.')
    
    return True

# Définition de l'environement Spark

In [5]:
role = get_execution_role()

# Configure Spark to use the SageMaker Spark dependency jars
jars = sagemaker_pyspark.classpath_jars()

classpath = ":".join(sagemaker_pyspark.classpath_jars())

# See the SageMaker Spark Github to learn how to connect to EMR from a notebook instance
spark = SparkSession.builder.config("spark.driver.extraClassPath", classpath)\
    .master("local[*]").getOrCreate()
    
spark

# Variables globales

In [6]:
# Chemin d'acces des données sur S3
# Seul un échantillon des données de base est utilisé

sample_img_dir = "s3a://p8muriel/test/*"
path_write = "s3a://p8muriel/test_deep_features.parquet"

# Pré-processing

## Chargement du modèle RESNET50

In [224]:
t = time.time()
model = ResNet50(weights='imagenet', pooling='avg', include_top = False) 
model.summary
print('...Temps de chargement du modèle : {}'.format(time.strftime("%H:%M:%S", time.gmtime(time.time()-t))))

...Temps de chargement du modèle : 00:00:01


## Chargement des données

Un échantillon des données sont stockées sur un bucket "p8muriel" sur S3. **Ni les chemins, ni les noms de fichiers ne doivent contenir des espaces.***  
-> faire un script préalable au chargement des données sur S3  

In [8]:
print("Chargement des images")
#Chargement des images dans un RDD sous forme binaire
t = time.time()
images = spark.sparkContext.binaryFiles(sample_img_dir)
print('...Temps de chargement des images : {}'.format(time.strftime("%H:%M:%S", time.gmtime(time.time()-t))))

Chargement des images
...Temps de chargement des images : 00:00:02


### Exemple de ligne du RDD pour deux images format  binaire

Ne pas executer dans le pré-traitement 

In [31]:
images.take(2)

[('s3a://p8muriel/test/AppleRed3/44_100.jpg',
  b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x01\x01\x01\x01\x02\x01\x01\x01\x02\x02\x02\x02\x02\x04\x03\x02\x02\x02\x02\x05\x04\x04\x03\x04\x06\x05\x06\x06\x06\x05\x06\x06\x06\x07\t\x08\x06\x07\t\x07\x06\x06\x08\x0b\x08\t\n\n\n\n\n\x06\x08\x0b\x0c\x0b\n\x0c\t\n\n\n\xff\xdb\x00C\x01\x02\x02\x02\x02\x02\x02\x05\x03\x03\x05\n\x07\x06\x07\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\xff\xc0\x00\x11\x08\x00d\x00d\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1\x15R\xd1\xf0$3br\x82\t\n\x16\x17\x18\x19\x1a%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\x83\x84\x85\x8

# Pre processing

## Conversion de l'image en array, extraction de la catégorie et du nom du fichier image

In [236]:
print("Conversion des images au format numerique, extraction des catégories")
t = time.time()
images_proc = images.map(lambda img: cat_image_np(img))
print('...Temps de traintement : {}'.format(time.strftime("%H:%M:%S", time.gmtime(time.time()-t))))

Conversion des images au format numerique, extraction des catégories
...Temps de traintement : 00:00:00


voici la transformation, à ne pas faire en pre-traitement sur un gros volume de données: affichage de la première ligne du RDD

In [238]:
images_proc.first()

Row(category='AppleRed3', fileName='44_100.jpg', image=[[[253.0, 255.0, 250.0], [255.0, 255.0, 253.0], [255.0, 255.0, 253.0], [255.0, 255.0, 255.0], [254.0, 255.0, 255.0], [250.0, 255.0, 255.0], [248.0, 255.0, 255.0], [246.0, 255.0, 255.0], [248.0, 255.0, 255.0], [251.0, 255.0, 255.0], [255.0, 253.0, 254.0], [255.0, 253.0, 254.0], [255.0, 252.0, 255.0], [254.0, 254.0, 255.0], [250.0, 255.0, 255.0], [248.0, 255.0, 255.0], [253.0, 249.0, 250.0], [255.0, 254.0, 255.0], [253.0, 253.0, 253.0], [252.0, 253.0, 255.0], [252.0, 255.0, 255.0], [252.0, 255.0, 255.0], [252.0, 255.0, 255.0], [251.0, 253.0, 250.0], [255.0, 255.0, 253.0], [255.0, 255.0, 253.0], [255.0, 254.0, 255.0], [249.0, 249.0, 251.0], [253.0, 254.0, 255.0], [250.0, 253.0, 255.0], [248.0, 255.0, 255.0], [248.0, 255.0, 255.0], [254.0, 253.0, 255.0], [250.0, 253.0, 255.0], [247.0, 255.0, 255.0], [244.0, 255.0, 255.0], [241.0, 255.0, 255.0], [247.0, 255.0, 255.0], [250.0, 255.0, 254.0], [254.0, 254.0, 254.0], [255.0, 253.0, 252.0], 

# Extraction des deep_feature avec RESNET50

In [241]:
images_f= images_proc.map(lambda y: deep_features(y))

voici la transformation, à ne pas faire en pre-traitement sur un gros volume de données: affichage de la première ligne du RDD

In [242]:
images_f.first()

Row(category='AppleRed3', feature=[0.0, 0.8664653301239014, 0.041645586490631104, 1.578285574913025, 0.030423052608966827, 0.0, 0.804176926612854, 0.0, 0.15386036038398743, 0.0, 0.6651325821876526, 0.4686932861804962, 0.0, 0.3588976263999939, 0.05741458386182785, 0.0, 0.0, 0.17537419497966766, 1.1800830364227295, 0.20829583704471588, 0.11190150678157806, 0.2419508546590805, 0.04129500687122345, 0.0030576884746551514, 0.16212108731269836, 0.0, 2.7885189056396484, 0.0694546326994896, 0.02245102822780609, 2.839057683944702, 0.01576802134513855, 0.0, 0.0, 0.08800065517425537, 1.2546027898788452, 0.07853154093027115, 0.01190580427646637, 4.3022260665893555, 0.0, 0.0, 0.1087731271982193, 0.0, 0.0, 0.03831753134727478, 0.05146761238574982, 0.235489621758461, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03773443028330803, 0.2856186032295227, 1.4525595903396606, 0.0, 0.3932586908340454, 0.0, 0.01983782649040222, 1.3679399490356445, 1.5961273908615112, 0.9825287461280823, 0.0, 0.35886090993881226, 8.8508567810058

# Enregistrement des données au format parquet

In [244]:
print("...conversion du rdd avec les deep features en spark dataframe")
t = time.time()  
df=images_f.toDF()
print('.....Temps de traitement toDF : {}'.format(time.strftime("%H:%M:%S", time.gmtime(time.time()-t))))

...conversion du rdd avec les deep features en spark dataframe
.....Temps de traitement toDF : 00:00:05


Pour information à ne pas realiser lors du pre_traitement, affichage de deux lignes du spark dataframe

In [245]:
df.show(2)

+---------+--------------------+----------+
| category|             feature|  fileName|
+---------+--------------------+----------+
|AppleRed3|[0.0, 0.866465330...|44_100.jpg|
|AppleRed3|[0.06866307556629...|45_100.jpg|
+---------+--------------------+----------+
only showing top 2 rows



In [223]:
df.show(2, False)

+---------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

schema du spark dataframe

In [222]:
df.printSchema()

root
 |-- category: string (nullable = true)
 |-- feature: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- fileName: string (nullable = true)



In [ ]:

print("...enregistrement format parquet")
t = time.time()  

write_dataframe_parquet(df,path_write)
print('.....Temps de traitement enregistrement parquet : {}'.format(time.strftime("%H:%M:%S", time.gmtime(time.time()-t))))

...enregistrement format parquet


In [ ]:
path_write

# Aperçu des données après réduction de dimension

## Ecriture en csv pour comparaison avec parquet

In [49]:
print("...enregistrement format csv")
t = time.time() 
df.toPandas().to_csv("s3a://p8muriel/test2_bow_file.csv", header=True)
print('.....Temps de traitement enregistrement csv : {}'.format(time.strftime("%H:%M:%S", time.gmtime(time.time()-t))))

...enregistrement format csv
.....Temps de traitement enregistrement csv : 00:10:02
